In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
data = pd.read_csv('./data/test_aus_data.csv').values.astype(floatX).T[np.newaxis][:,1:]/100

In [3]:
# data = data[:,1:] - data[:,:-1]

In [4]:
stds = (data[0,1:] - data[0,:-1]).std(axis=0)
print(stds)
data /= stds

[0.0186606  0.01934265 0.00918617]


In [5]:
data = np.concatenate([data[:,1:], data[:,:-1]], axis=-1)
data.shape

(1, 36, 6)

In [6]:
np.any(np.isnan(data))

False

In [7]:
NUM_STEPS = data.shape[1]

with tf.variable_scope('walk_ord'):
    s1_prior_d = LogNormal(1, mu=math.log(0.01), sigma=6.)

    with tf.variable_scope('s1_inference', dtype=floatX):
        mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(s1_prior_d.mu))
        logsigma = tf.get_variable('logsigma', shape=[1], 
                                   initializer=tf.constant_initializer(min(math.log(s1_prior_d.sigma), -1.)))
        sigma = tf.exp(logsigma)
        s1_d = LogNormal(1, mu=mu, sigma=sigma)
    
    s1 = s1_d.sample()
    tf.summary.scalar('s1_ord', s1[0])
    tf.add_to_collection('logdensities', s1_d.logdens(s1))
    
    s1_prior = s1_prior_d.logdens(s1)
    tf.add_to_collection('priors', s1_prior)
    
#distribution for shock increase factor
with tf.variable_scope('walk_shock'):
    incr_prior_d = LogNormal(1, mu=math.log(1.), sigma=3.)

    with tf.variable_scope('incr_inference', dtype=floatX):
        mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(incr_prior_d.mu))
        logsigma = tf.get_variable('logsigma', shape=[1], 
                                   initializer=tf.constant_initializer(min(math.log(incr_prior_d.sigma), -1.)))
        sigma = tf.exp(logsigma)
        incr_d = LogNormal(1, mu=mu, sigma=sigma)

    incr = incr_d.sample()
    tf.add_to_collection('logdensities', incr_d.logdens(incr))
    
    incr_prior = incr_prior_d.logdens(incr)
    tf.add_to_collection('priors', incr_prior)

    incr += 1.3
    s_shock = s1*incr
    tf.summary.scalar('s1_shock', s_shock[0])
    
s0 = 80.
dim = [3,3*2+1]

K = dim[0] * dim[1]

PWalk = NormalRW(dim=None, sigma0=s0, sigma=s1, name='OrdWalk')
PWalk_shock = NormalRW(dim=None, sigma0=s0, sigma=s_shock, name='ShockWalk')

In [8]:
#model coefficients
gvar = GVAR(dim=dim[0]*dim[1], len=NUM_STEPS, name='coef_rw_inference')
outputs = gvar.sample()
# with tf.variable_scope('coefs_inference', dtype=floatX):
#     gvar = DFlow([LinearChol(NUM_STEPS*dim[0]*dim[1], name='coef_rw_inference', use_bias=False)], init_sigma=0.02)
#     outputs = tf.reshape(gvar.output, [1,NUM_STEPS, dim[0]*dim[1]])
#     outputs = tf.cumsum(outputs, axis=1)
#     mu = tf.get_variable('mu', [1,NUM_STEPS,dim[0]*dim[1]])
#     outputs += mu

tf.add_to_collection('logdensities', gvar.logdens)

In [9]:
outputs

<tf.Tensor 'coef_rw_inference_1/VAR/strided_slice_36:0' shape=(1, 36, 21) dtype=float64>

In [10]:
prior_ord = PWalk.logdens(outputs, reduce=False)
prior_shock = PWalk_shock.logdens(outputs, reduce=False)

In [11]:
prior_ord, prior_shock

(<tf.Tensor 'OrdWalk_2/concat:0' shape=(1, 36) dtype=float64>,
 <tf.Tensor 'ShockWalk_2/concat:0' shape=(1, 36) dtype=float64>)

In [12]:
with tf.variable_scope('Shock_Distr', reuse=tf.AUTO_REUSE):
    shock_init_d = Normal([1,8])
    shock_init = shock_init_d.sample()
    
    tf.add_to_collection('logdensities', shock_init_d.logdens(shock_init))
    
    shock_d = flows.dvae.RMultinomial([data.shape[1], 2], name='shock_distr')
    shocks_soft_uncond, shocks_soft_cond, (shocks_hard, shocks_hard_logp, encoding_entropy) =\
    shock_d.encode(shock_init)
    
    shocks_hard_logp = tf.reduce_sum(shocks_hard_logp)

In [13]:
shock_d.temp

<tf.Tensor 'Shock_Distr/shock_distr/controls/Exp:0' shape=(36, 1) dtype=float64>

In [14]:
shocks_hard

<tf.Tensor 'Shock_Distr/shock_distr_1/encoder/latent_inf/Cast_1:0' shape=(1, 36, 2) dtype=float64>

In [15]:
def create_hard_shock_prior(num_shocks):
    def c(n,k):
        from scipy.special import binom
        return tf.py_func(lambda x,y: int(binom(int(x),int(y))), [n, k], tf.int64)
    
    with tf.variable_scope('shocks_prior'):
        spd = tf.contrib.distributions.Poisson(rate=NUM_STEPS/10.)
        denum = tf.cast(c(NUM_STEPS, num_shocks), floatX)
        return tf.cast(spd.log_prob(tf.cast(num_shocks, tf.float32)), floatX) - tf.log(denum)

In [16]:
def create_soft_shock_prior(num_shocks):
#     return 0.
    lower = tf.cast(tf.floor(num_shocks), floatX)
    upper = lower + 1.
    lh = create_hard_shock_prior(lower)
    uh = create_hard_shock_prior(upper)
    prop = num_shocks-lower
    return lh*(1.-prop) + uh*prop

In [17]:
obs_d_prior = LogNormal(dim=None, mu=math.log(0.5), sigma=0.5)

with tf.variable_scope('obs_d_inference', dtype=floatX):
    mu = tf.get_variable('mu', shape=[1], initializer=tf.constant_initializer(math.log(0.5)))
    logsigma = tf.get_variable('logsigma', shape=[1], initializer=tf.constant_initializer(-5))
    sigma = tf.exp(logsigma)
    
    obs_d_post = LogNormal(1, mu=mu, sigma=sigma)
    obs_ds = obs_d_post.sample()
    
    obs_ds_logdens = obs_d_post.logdens(obs_ds)
    tf.add_to_collection('logdensities', obs_ds_logdens)
    tf.add_to_collection('priors', obs_d_prior.logdens(obs_ds))
    
    tf.summary.scalar('observ_sigma', obs_ds[0])

In [18]:
def create_param_walk_prior(shocks):
    with tf.name_scope('RW_prior'):
        tmp = tf.cast(shocks[:,:,1], floatX)
        print(tmp, prior_shock)
        prior = tmp*prior_shock + (1-tmp)*prior_ord
        print(prior)
        prior = tf.reduce_sum(prior) 
        return prior

In [19]:
def predict(observable_mask):
    out = tf.reshape(outputs, [NUM_STEPS, dim[0], dim[1]])
    
    def step(prev, x):
        mask = x[0]
        prev_pred = tf.where(mask, x[1], prev)[tf.newaxis]
        params = x[2]
        
        d0 = params[:,:dim[0]]
        d1 = params[:,dim[0]:2*dim[0]]
        
        pp1 = prev_pred[:,:dim[0]]
        pp0 = prev_pred[:,dim[0]:2*dim[0]]
        
        new_pred = tf.matmul(pp0, d0)[0] + tf.matmul(pp1, d1)[0]+ params[:,-1] + pp1[0]
        new_pred = tf.concat([new_pred, pp1[0]], axis=0)
        return new_pred
    
    ar = tf.scan(step, [observable_mask, data[0], out], initializer=tf.zeros([2*dim[0]], dtype=floatX))
    return ar

In [20]:
data.shape[1]

36

In [21]:
stop_time = tf.placeholder_with_default(NUM_STEPS, ())
predict_mask = tf.range(0, data.shape[1], dtype=tf.int32) < stop_time
#используем в предсказаниях ровно stop_time последних наблюдений

In [22]:
preds = predict(predict_mask)

In [23]:
preds

<tf.Tensor 'scan/TensorArrayStack/TensorArrayGatherV3:0' shape=(36, 6) dtype=float64>

In [24]:
obs_d = Normal(dim=None, sigma=obs_ds, mu=0)

diffs = preds[:-1] - data[0,1:]
diffs = diffs[:,:dim[0]]

logl = obs_d.logdens(diffs, reduce=False)
logl *= tf.cast(predict_mask[:-1], floatX)[:,tf.newaxis]

logl = tf.reduce_sum(logl)
tf.add_to_collection('priors', logl)

In [25]:
diffs

<tf.Tensor 'strided_slice_1:0' shape=(35, 3) dtype=float64>

In [26]:
tf.summary.scalar('logl', logl)

<tf.Tensor 'logl:0' shape=() dtype=string>

In [27]:
def create_kl(shocks, shocks_prior=0.):
    logdens = tf.reduce_sum(tf.get_collection('logdensities'))
    prior = tf.reduce_sum(tf.get_collection('priors'))
    
    param_walk_prior = create_param_walk_prior(shocks)
    KLd = -prior + logdens - shocks_prior - param_walk_prior
    KLd /= NUM_STEPS*dim[0]*dim[1]
    return KLd

In [28]:
tf.get_collection('logdensities')

[<tf.Tensor 'walk_ord/LogNormal_2/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'walk_shock/LogNormal_2/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'coef_rw_inference_1/VAR/logdens/add:0' shape=() dtype=float64>,
 <tf.Tensor 'Shock_Distr/Normal_2/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'obs_d_inference/LogNormal_2/Sum:0' shape=() dtype=float64>]

In [29]:
tf.get_collection('priors')

[<tf.Tensor 'walk_ord/LogNormal_3/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'walk_shock/LogNormal_3/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'obs_d_inference/LogNormal_3/Sum:0' shape=() dtype=float64>,
 <tf.Tensor 'Sum:0' shape=() dtype=float64>]

In [30]:
shocks_soft_cond

<tf.Tensor 'Shock_Distr/shock_distr_1/encoder/latent_inf/Reshape_4:0' shape=(1, 36, 2) dtype=float64>

In [31]:
num_shocks = tf.reduce_sum(shocks_soft_uncond[:,:,1:])
ssp = create_soft_shock_prior(num_shocks) #- encoding_entropy
soft_kl_uncond = create_kl(shocks_soft_uncond, shocks_prior=ssp)

num_shocks = tf.reduce_sum(shocks_soft_cond[:,:,1:])
ssp = create_soft_shock_prior(num_shocks) #- encoding_entropy
soft_kl_cond = create_kl(shocks_soft_cond, shocks_prior=ssp)

num_shocks = tf.reduce_sum(shocks_hard[:,:,1:])
hard_shocks_prior = create_hard_shock_prior(num_shocks) - shocks_hard_logp
hard_kl = create_kl(shocks_hard, shocks_prior=hard_shocks_prior)

Tensor("RW_prior/strided_slice:0", shape=(1, 36), dtype=float64) Tensor("ShockWalk_2/concat:0", shape=(1, 36), dtype=float64)
Tensor("RW_prior/add:0", shape=(1, 36), dtype=float64)
Tensor("RW_prior_1/strided_slice:0", shape=(1, 36), dtype=float64) Tensor("ShockWalk_2/concat:0", shape=(1, 36), dtype=float64)
Tensor("RW_prior_1/add:0", shape=(1, 36), dtype=float64)
Tensor("RW_prior_2/strided_slice:0", shape=(1, 36), dtype=float64) Tensor("ShockWalk_2/concat:0", shape=(1, 36), dtype=float64)
Tensor("RW_prior_2/add:0", shape=(1, 36), dtype=float64)


In [32]:
tf.summary.scalar('num_shocks', num_shocks)

<tf.Tensor 'num_shocks:0' shape=() dtype=string>

In [33]:
shocks_hard_logp

<tf.Tensor 'Shock_Distr/Sum:0' shape=() dtype=float64>

In [34]:
relax_cond = shock_d.build_relax(tf.reshape(shocks_soft_cond, [1,-1]))
relax_uncond = shock_d.build_relax(tf.reshape(shocks_soft_uncond, [1,-1]))

In [35]:
soft_kl_cond

<tf.Tensor 'truediv_1:0' shape=<unknown> dtype=float64>

In [36]:
eta = shock_d.eta

soft_control_cond = eta*soft_kl_cond + relax_cond
soft_control_uncond = eta*soft_kl_uncond - eta*soft_kl_cond - relax_cond + relax_uncond

stoped_grad = hard_kl - soft_control_cond# - nvil_baseline

target = shocks_hard_logp*stoped_grad + soft_control_uncond
target = tf.reduce_mean(target)

In [37]:
soft_control_uncond

<tf.Tensor 'add_8:0' shape=<unknown> dtype=float64>

In [38]:
shock_vars = shock_d.get_encoder_vars()

In [39]:
shock_vars_grad = tf.gradients(target, shock_vars, stop_gradients=stoped_grad)

In [40]:
controls = shock_d.get_control_vars()
cv_loss = shock_d.build_control_loss(shock_vars_grad)

tf.summary.scalar('cv_loss', cv_loss)

cv_gradients = tf.gradients(cv_loss, controls)

In [41]:
rest_vars = [x for x in tf.global_variables() if not (x in controls or x in shock_vars)]
rest_gradients = tf.gradients(hard_kl, rest_vars)

In [42]:
rest_vars

[<tf.Variable 'walk_ord/s1_inference/mu:0' shape=(1,) dtype=float64_ref>,
 <tf.Variable 'walk_ord/s1_inference/logsigma:0' shape=(1,) dtype=float64_ref>,
 <tf.Variable 'walk_shock/incr_inference/mu:0' shape=(1,) dtype=float64_ref>,
 <tf.Variable 'walk_shock/incr_inference/logsigma:0' shape=(1,) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_0/lowerd:0' shape=(231,) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_0/ldiag:0' shape=(21,) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_1/lowerd:0' shape=(231,) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_1/ldiag:0' shape=(21,) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_1/W:0' shape=(21, 21) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_2/lowerd:0' shape=(231,) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_2/ldiag:0' shape=(21,) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_2/W:0' shape=(21, 21) dtype=float64_ref>,
 <tf.Variable 'coef_rw_inference/lc_3/lowerd:0' 

In [43]:
all_vars = shock_vars + rest_vars 
grads = shock_vars_grad + rest_gradients
main_updates = [(g,x) for x,g in zip(all_vars, grads)]
control_updates = [(g,x) for x,g in zip(controls, cv_gradients)]

In [44]:
tf.summary.scalar('KLd', hard_kl)

<tf.Tensor 'KLd:0' shape=() dtype=string>

In [45]:
lr = tf.Variable(0.0001)
main_opt = tf.train.AdamOptimizer(lr).apply_gradients(main_updates)
cont_opt = tf.train.AdamOptimizer(0.0003).apply_gradients(control_updates)

In [46]:
control_updates

[(<tf.Tensor 'gradients_1/Shock_Distr/shock_distr/controls/Exp_grad/mul:0' shape=(36, 1) dtype=float64>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/pretemp:0' shape=(36, 1) dtype=float64_ref>),
 (<tf.Tensor 'gradients_1/Shock_Distr/shock_distr/controls/Exp_1_grad/mul:0' shape=() dtype=float64>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/preeta:0' shape=() dtype=float64_ref>),
 (<tf.Tensor 'gradients_1/AddN_30:0' shape=(72, 128) dtype=float64>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/RELAX/FCN/d0/W:0' shape=(72, 128) dtype=float64_ref>),
 (<tf.Tensor 'gradients_1/AddN_29:0' shape=(1, 128) dtype=float64>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/RELAX/FCN/d0/b:0' shape=(1, 128) dtype=float64_ref>),
 (<tf.Tensor 'gradients_1/AddN_24:0' shape=(128, 64) dtype=float64>,
  <tf.Variable 'Shock_Distr/shock_distr/controls/RELAX/FCN/d1/W:0' shape=(128, 64) dtype=float64_ref>),
 (<tf.Tensor 'gradients_1/AddN_22:0' shape=(1, 64) dtype=float64>,
  <tf.Variable 'Shock_Dis

In [47]:
opt = [main_opt, cont_opt]

In [48]:
sess = tf.InteractiveSession()

In [49]:
# !rm -R /tmp/tfdbg

In [50]:
!mkdir /tmp/tfdbg
writer = tf.summary.FileWriter('/tmp/tfdbg/proper_prior_var_aus_rate10_2nd_order_fdif-init-uninformative-better_init')

mkdir: cannot create directory ‘/tmp/tfdbg’: File exists


In [51]:
writer.add_graph(tf.get_default_graph())

In [52]:
tf.global_variables_initializer().run()

In [53]:
hard_kl.eval()

1.0465753006116203

In [54]:
cv_gradients[0]

<tf.Tensor 'gradients_1/Shock_Distr/shock_distr/controls/Exp_grad/mul:0' shape=(36, 1) dtype=float64>

In [55]:
# a = []
# for _ in range(1000):
#     a.append(sess.run(cv_gradients[0]))
# np.mean(a, axis=0)

In [56]:
np.set_printoptions(precision=3)

In [57]:
# PWalk.inverse_sigma.eval()[:3,:3]

In [58]:
out = tf.reshape(outputs, [NUM_STEPS, dim[0], dim[1]])

In [59]:
epoch = 0

In [60]:
all_sum = tf.summary.merge_all()

In [61]:
def check_ppc(timestep):
    n = 1000
    all_preds = []
    for _ in range(n):
        tp = preds.eval({stop_time:timestep-1})
        tp = tp[timestep-1]
        all_preds.append(tp)
    all_preds = np.array(all_preds).mean(axis=0)
#     print(xs.shape)
    return np.sqrt(np.mean((all_preds - xs[0,timestep])**2))

In [62]:
# shock_d.logits.eval()

In [ ]:
%%time
for epoch in range(epoch, epoch+2000):
    for i in range(1000):
        l, _ = sess.run([hard_kl, opt], {phase:True})
        if i % 30 == 0:
            s = all_sum.eval({phase:True})
            writer.add_summary(s, global_step=epoch*1000 + i)
    print(l)

0.6593520513854791
0.48980428421510736
0.6157712921849169
0.5604748373390684
0.4401577630028026
0.5303250816609485
0.47384806572831356
0.4265251396979143
0.41014568929714074
0.4020585223844268
0.4329198850947279
0.4207747100435322
0.415293716698064
0.4990400439572765
0.40241195581257966
0.3877008406706458
0.38884711987086323
0.42985566173540846
0.38910053083466584
0.41453509091091245
0.4218960331471942
0.37999891922956486
0.41709015451278036
0.44611120695366485
0.3971691702635315
0.3871439930282502
0.38577239270439084
0.3751705654935832
0.3817299819026561
0.4365155169497353
0.38333789723452855
0.3816898699006302
0.38090770626627274
0.41287606389277853
0.41036468737670584
0.370237169359587
0.36906400559362657
0.37142561411517955
0.3723960511166217
0.36638947404859573
0.3635846936321546
0.38323824931531125
0.38791402798397895
0.35891285155611796
0.37469814082401615
0.37083717142106837
0.3607307585189925
0.3610090318116985
0.36618673954355696
0.37018952935973815
0.3660112089052924
0.36917

0.34643229741895537
0.3477995001659494
0.36347185583447394
0.3471224326262723
0.37134954085371086
0.3527506071672366
0.3426077058126713
0.35257403541788246
0.35851367552586827
0.3701869961004907
0.3479796666044672
0.3371398044256767
0.35025479605439164
0.36928172993692493
0.3555268256817938
0.3450831748269231
0.3552531318584747
0.3528211313675508
0.3461667046047118
0.3428461585105198
0.3463790104123785
0.37390350882686013
0.3673722473533998
0.3493975899206122
0.34593076486567587
0.3507846625963262
0.3480239945946262
0.3653543033200424
0.35169514461759543
0.35890188074338925
0.3737002714594398
0.35997652165918853
0.3740811019968078
0.3376174347248269
0.3673609617226729
0.34178714014525985
0.34539490028101916
0.3428132032717121
0.3440157136356576
0.3452389203768013
0.35162834089492173
0.3442881821502688
0.3407253859297623
0.33702857933331826
0.3561092741419673
0.3657169348124615
0.34196648533770113
0.3487522309656159
0.35231772107876186
0.3703762032383871
0.34599341342334355
0.3529844191

In [ ]:
lr.assign(0.00003).eval()

In [ ]:
ss = []
for _ in range(40000):
    s = shocks_hard.eval()[:,:,1].astype('float')
    ss.append(s)
ss = np.array(ss)
print(ss.mean(axis=0))

In [ ]:
x = range(2016-data.shape[1],2016)
plt.plot(x, ss.mean(axis=0)[0][:])
plt.show()

In [ ]:
ss = []
aa = []
for _ in range(3000):
    a = np.linalg.inv(PWalk.inverse_sigma.eval())
    ss.append(np.sqrt(np.diag(a)))
    aa.append(a)
ss = np.array(ss)
aa = np.array(aa)

In [ ]:
aa.mean(axis=0)[:5,:5]

In [ ]:
np.mean(ss, axis=0), np.std(ss, axis=0)

In [ ]:
# sns.kdeplot(ss[:,1], ss[:,0])
# plt.show()

In [ ]:
samples = []
for _ in range(3000):
    s = out.eval({phase:True})
    samples.append(s)
samples = np.array(samples)

In [ ]:
t = 10
sns.kdeplot(samples[:,t,2,0], samples[:,t,0,3])
plt.show()

In [ ]:
t = 10
# params[t]

In [ ]:
np.mean(samples[:,t], axis=0)

In [ ]:
np.std(samples[:,t], axis=0)

In [ ]:
from sklearn.neighbors.kde import KernelDensity
def score(s1, s2):
    bw = 0.1
    s1 = s1[:,np.newaxis]
    s2 = s2[:,np.newaxis]
    e1 = KernelDensity(bandwidth=bw).fit(s1)
    e2 = KernelDensity(bandwidth=bw).fit(s2)
    
    e1e2 = e1.score(s1) - e2.score(s1)
    e2e1 = e2.score(s2) - e1.score(s2)
    dist = (e1e2/len(s1) + e2e1/len(s2))/2
    return dist

In [ ]:
def get_cdf(ts):
    return lambda x: (ts <= x).mean()
def score(s1,s2):
    c1 = get_cdf(s1)
    c2 = get_cdf(s2)

    m = 0
    for s in np.concatenate([s1, s2]):
        r = np.abs(c1(s) - c2(s))
        if r > m:
            m = r
    return m

In [ ]:
score(s1,s2)

In [ ]:
dists = []
for t1 in range(samples.shape[1] - 1):
    np.random.seed(1234)
    print(t1)
    t2 = t1 + 1
    s1 = samples[:,t1,0,0]
    s2 = samples[:,t2,0,0]
    ss = []
    for _ in range(10):
        s1_ = np.random.choice(s1, size=8000)
        s2_ = np.random.choice(s2, size=8000)
        m = score(s1_,s2_)
        ss.append(m)
    dists.append(ss)
dists = np.array(dists).T

## KDE + KL

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

## Kolmogorov-Smirnov

In [ ]:
sns.tsplot(dists, ci=[50, 95])
plt.show()

In [ ]:
tf.pow()